In [2]:
'''
Plant disease
AlternariaBoltch, AppleScab, BlackRot, CedarAppleRust, Rust

Only classifying between healthy or not
'''

'\nPlant disease\nAlternariaBoltch, AppleScab, BlackRot, CedarAppleRust, Rust\n\nOnly classifying between healthy or not\n'

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import RMSprop
import os

%matplotlib inline

In [4]:
# check image size
import cv2

im = cv2.imread('Plant disease dataset/Image-level annotation/train/Apple_healthy/Healthy_0.jpg')

print(type(im))
# <class 'numpy.ndarray'>

print(im.shape) # (256, 256, 3)
print(type(im.shape))

<class 'numpy.ndarray'>
(256, 256, 3)
<class 'tuple'>


In [5]:
train_gen = ImageDataGenerator(rescale=1.0/256.0, zoom_range=0.4, rotation_range=30,
                               horizontal_flip=True, vertical_flip=True, shear_range=0.3, fill_mode='nearest',
                               width_shift_range=0.2, height_shift_range=0.2, validation_split=0.2)
valid_gen = ImageDataGenerator(rescale=1.0/256.0)

In [6]:
train_dir = 'Plant disease dataset/Image-level annotation/train'
val_dir = 'Plant disease dataset/Image-level annotation/valid'

train_dataset = train_gen.flow_from_directory(train_dir, target_size=(256, 256), color_mode="rgb",
                                              class_mode='binary', batch_size=32)
validation_dataset = valid_gen.flow_from_directory(val_dir, target_size=(256, 256), color_mode="rgb",
                                                   class_mode='binary', batch_size=32)

Found 27892 images belonging to 2 classes.
Found 9700 images belonging to 2 classes.


In [15]:
train_dataset.class_indices

{'Apple_disease': 0, 'Apple_healthy': 1}

In [8]:
validation_dataset.class_indices

{'Apple_disease': 0, 'Apple_healthy': 1}

In [9]:
# from GPT, doesn't work, m8 try later
# import glob
# import os
# import pandas as pd
# from sklearn.model_selection import train_test_split

# all_images = glob.glob(os.path.join(train_dir, '*', '*.jpg'))

# # Extract labels based on subdirectory names
# # I better remember this code so i can classify every disease instead
# labels = [os.path.basename(os.path.dirname(img)) for img in all_images]

# # Split the dataset into train and test sets
# train_images, test_images, train_labels, test_labels = train_test_split(all_images, labels, test_size=0.2, random_state=42, stratify=labels)

# # Create Pandas DataFrames for train and test sets
# train_df = pd.DataFrame({'filename': train_images, 'label': train_labels})
# test_df = pd.DataFrame({'filename': test_images, 'label': test_labels})

# # Create separate generators for train and test datasets
# train_dataset = train_gen.flow_from_dataframe(train_df, directory=train_dir, x_col='filename', y_col='label',
#                                               target_size=(256, 256), color_mode='rgb', class_mode='binary', batch_size=64)

# test_dataset = valid_gen.flow_from_dataframe(test_df, directory=train_dir, x_col='filename', y_col='label',
#                                              target_size=(256, 256), color_mode='rgb', class_mode='binary', batch_size=64)


In [10]:
# Test using 11 layer
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=32, activation='relu', input_shape=(256, 256, 3), padding='same', kernel_size=(3, 3)),
    tf.keras.layers.Conv2D(filters=32, activation='relu', padding='same', kernel_size=(3, 3)),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(filters=64, activation='relu', padding='same', kernel_size=(3, 3)),
    tf.keras.layers.Conv2D(filters=64, activation='relu', padding='same', kernel_size=(3, 3)),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(filters=128, activation='relu', padding='same', kernel_size=(3, 3)),
    tf.keras.layers.Conv2D(filters=128, activation='relu', padding='same', kernel_size=(3, 3)),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(filters=256, activation='relu', padding='same', kernel_size=(3, 3)),
    tf.keras.layers.Conv2D(filters=256, activation='relu', padding='same', kernel_size=(3, 3)),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    # Flatten
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='sigmoid'),
])

monitor = EarlyStopping(monitor='val_accuracy',
                        patience=2,
                        verbose=1,
                        mode='max',
                        restore_best_weights=True)

In [11]:
model.compile(optimizer=RMSprop(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.AUC(name='accuracy')])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 256, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 32)     0         
 )                                                               
                                                                 


 conv2d_2 (Conv2D)           (None, 128, 128, 64)      18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 128, 128, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 64, 64, 64)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 64, 64, 128)       73856     
                                                                 
 conv2d_5 (Conv2D)           (None, 64, 64, 128)       147584    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 32, 32, 128)      0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 32, 32, 256)       295168    
          

In [12]:
# # 24. Train the model
# model.fit(train_dataset, epochs=1, validation_data=validation_dataset)

# # Fine-tune the model
# for layer in base_model.layers[-4:]:
#     layer.trainable = True

In [13]:
# # Defining callbacks and save best model
# filepath = 'Output/best_weights.hdf5'
# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', mode='max', save_best_only=True, verbose=1)

# callback_list = [monitor, checkpoint]

# model.compile(optimizer=RMSprop(learning_rate=0.001),
#               loss='binary_crossentropy',
#               metrics=[tf.keras.metrics.AUC(name='accuracy')])
# history = model.fit(train_dataset, 
#                     validation_data=validation_dataset,
#                     epochs=10,
#                     verbose=1,
#                     callbacks=callback_list)

Epoch 1/10
  4/872 [..............................] - ETA: 55:44 - loss: 15.2521 - accuracy: 0.4283

KeyboardInterrupt: 